In [69]:
# Load library
import fairsearchcore as fsc
import pandas as pd
dataframen = pd.read_csv("ONE_samlet_preds_index_ny_WO.csv")
dataframen

,Unnamed: 0,Idx,Gender_bins,Preds
0,0,12016,[0],93.777222
1,1,11864,[1],93.685806
2,2,8893,[0],93.681221
3,3,4900,[0],93.660019
4,4,12018,[1],93.660019
...,...,...,...,...
4091,4091,9028,[1],18.230223
4092,4092,1043,[0],17.869230
4093,4093,4014,[1],17.439417
4094,4094,8687,[0],15.629829


In [2]:
nydf = dataframen.iloc[:600]
men = len(nydf[nydf.Gender_bins == 0])
women = len(nydf[nydf.Gender_bins != 0])

women/(women+men)

0.36

In [21]:

k = 400 # number of topK elements returned (value should be between 10 and 400)
p = 0.4 # proportion of protected candidates in the topK elements (value should be between 0.02 and 0.98) 
alpha = 0.1 # significance level (value should be between 0.01 and 0.15)

# create the Fair object 
fair = fsc.Fair(k, p, alpha)

# create an mtable using alpha unadjusted
mtable = fair.create_unadjusted_mtable()

# analytically calculate the fail probability
# analytical = fair.compute_fail_probability(mtable)

# create an mtable using alpha adjusted
mtable = fair.create_adjusted_mtable()

# # again, analytically calculate the fail probability
# analytical = fair.compute_fail_probability(mtable)

In [22]:
# dataframen.Gender_bins.bool
dataframen["Gender_bins"] = dataframen["Gender_bins"].astype('bool')

In [23]:
from fairsearchcore.models import FairScoreDoc

liste = []
for i in range(400):
    # FairScoreDoc(dataframen.Idx[i],dataframen.Preds[i],dataframen.Gender_bins[i])
    liste.append(FairScoreDoc(dataframen.Idx[i],dataframen.Preds[i],dataframen.Gender_bins[i]))

org_ranking = liste

In [24]:
listeM = []
dataframenM = dataframen[dataframen.Gender_bins == False]

for i in range(len(dataframenM)):
    FairScoreDoc(dataframenM.iloc[i].Idx,dataframenM.iloc[i].Preds,dataframenM.iloc[i].Gender_bins)
    listeM.append(FairScoreDoc(dataframenM.iloc[i].Idx,dataframenM.iloc[i].Preds,dataframenM.iloc[i].Gender_bins))


In [25]:
listeF = []
dataframenF = dataframen[dataframen.Gender_bins == True]

for i in range(len(dataframenF)):
    FairScoreDoc(dataframenF.iloc[i].Idx,dataframenF.iloc[i].Preds,dataframenF.iloc[i].Gender_bins)
    listeF.append(FairScoreDoc(dataframenF.iloc[i].Idx,dataframenF.iloc[i].Preds,dataframenF.iloc[i].Gender_bins))

        


In [26]:
listeF

[<FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc

In [9]:
len(dataframenF)

333056

In [10]:
len(dataframenM)

486144

In [11]:
len(dataframenF)/(len(dataframenF)+len(dataframenM))

0.4065625

In [6]:
fair.is_fair(org_ranking)

False

In [7]:
re_ranked = fair.re_rank(org_ranking)
re_ranked[1]
# len(re_ranked)
# re_ranked = re_ranked[:300]
# len(re_ranked)

<FairScoreDoc [Nonprotected]>

In [15]:
org_ranking

[<FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Nonprotected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Protected]>,
 <FairScoreDoc [Nonprotected]>,


In [19]:
listeF

[]

In [42]:
from fairsearchcore.re_ranker import fair_top_k
res = fair_top_k(k,listeF, listeM, mtable)

In [29]:
fair.is_fair(res)

True

In [32]:
re_ranked_fair = fair.re_rank(res)
fair.is_fair(re_ranked_fair)

True

In [43]:
kon = []
for i in range(20):
    kon.append(res[i].is_protected + 0)
    
sum(kon)/len(kon)
    
    

0.15

In [41]:
kon = []
for i in range(20):
    kon.append(re_ranked_fair[i].is_protected + 0)
    
sum(kon)/len(kon)
    
    

0.15

In [45]:
kon = []
for i in range(20):
    kon.append(org_ranking[i].is_protected + 0)
    
sum(kon)/len(kon)
    
    

0.15

## NY FA*IR funktion: Maximize utility while maximizing fairness. 




In [70]:
df_try = dataframen#[:10000]
df_try

,Unnamed: 0,Idx,Gender_bins,Preds
0,0,12016,[0],93.777222
1,1,11864,[1],93.685806
2,2,8893,[0],93.681221
3,3,4900,[0],93.660019
4,4,12018,[1],93.660019
...,...,...,...,...
4091,4091,9028,[1],18.230223
4092,4092,1043,[0],17.869230
4093,4093,4014,[1],17.439417
4094,4094,8687,[0],15.629829


In [71]:
# If WO run this
# df_try['Gender_bins'] = df_try['Gender_bins'].str.strip('[]').astype(int)
# df_try

,Unnamed: 0,Idx,Gender_bins,Preds
0,0,12016,0,93.777222
1,1,11864,1,93.685806
2,2,8893,0,93.681221
3,3,4900,0,93.660019
4,4,12018,1,93.660019
...,...,...,...,...
4091,4091,9028,1,18.230223
4092,4092,1043,0,17.869230
4093,4093,4014,1,17.439417
4094,4094,8687,0,15.629829


In [72]:
def our_FAIR(df, prop, n_prefix, k):
    
    """" 
    Our fair algorithm inspired by Zehlike's FA*IR:
    df is predicted df. prop is desired proportion. n_prefix is desired prop for each n prefix, 
    k is top k ranking from df of length N. 
    
    Output is 'result' of a fair top-k ranking with desired proportion for the whole df and in every prefix
    
    """
    
    #we subtract by 0, because we are using 0-index
    n_prefix = n_prefix-1
    
    #split df in two df groups: protected and non-protected and ensure each of them are sorted
    #in descending order by Preds scores
    
    #protected (1=Female)
    dfp = df[df.Gender_bins == 1].sort_values(by='Preds', ascending=False).reset_index()
    #non-protected
    dfpn = df[df.Gender_bins == 0].sort_values(by='Preds', ascending=False).reset_index()
    
    #dfnew with of the top-k fair sorting of org. df
    #new df
    dfnew = pd.DataFrame(columns = df.columns)
    
    #print OBS if it's not possible to achieve the given prop for each n prefix for the whole top-k
    
    #for-loop for creating ranking in batches of prefix_n through k
    
    #desired number of protected candidates in each n batch
    prop_ok = int(n_prefix*prop)
    prop_pn = int(n_prefix - prop_ok)
    
    #index in each dataframe
    dfp_Idx = 0
    dfpn_Idx = 0
    
    for i in range(int(k/n_prefix)): #loop over n_prefixes in length k
        p_Idx = 0
        pn_Idx = 0
        for j in range(n_prefix):
            if dfp.Preds.loc[dfp_Idx] > dfpn.Preds[dfpn_Idx]:
                dfnew = dfnew.append(dfp.loc[dfp_Idx])
                p_Idx += 1
                dfp_Idx += 1
            else:
                dfnew = dfnew.append(dfpn.loc[dfpn_Idx])
                pn_Idx += 1
                dfpn_Idx += 1

            #if the proportion of nonprotected group is achieved within the prefix, add rest as protected 
            if pn_Idx == prop_pn:
                dfnew = dfnew.append(dfp.loc[dfp_Idx : dfp_Idx + (prop_ok-p_Idx)])
                dfp_Idx += (prop_ok-p_Idx)
                break
            #if the proportion of protected group is achieved within the prefix, add rest as non-protected
            elif p_Idx == prop_ok:
                dfnew = dfnew.append(dfpn.loc[dfpn_Idx : dfpn_Idx + (prop_pn-pn_Idx) ])
                dfpn_Idx += (prop_pn-pn_Idx)
                break

        dfnew = dfnew.drop(["index"], axis=1).reset_index()
                
    return dfnew



In [73]:
ress = our_FAIR(df_try, 0.40, 30, 500)
ress[:15]

,index,Unnamed: 0,Idx,Gender_bins,Preds
0,0,0.0,12016.0,0.0,93.777222
1,1,1.0,11864.0,1.0,93.685806
2,2,2.0,8893.0,0.0,93.681221
3,3,3.0,4900.0,0.0,93.660019
4,4,4.0,12018.0,1.0,93.660019
5,5,5.0,11651.0,0.0,93.405167
6,6,6.0,4874.0,0.0,93.268967
7,7,7.0,12028.0,0.0,93.268967
8,8,8.0,8905.0,0.0,92.923935
9,9,9.0,11800.0,0.0,92.923935


Check ratio of women out of total

In [74]:
# sum(ress.Gender_bins)/len(ress)
print(ress.Gender_bins.value_counts())
print(len(ress[ress.Gender_bins == 1]))
print(len(ress))
# print(len(ress[ress.Gender_bins == 1])/len(ress))
print('{0:.05f}'.format(len(ress[ress.Gender_bins == 1])/len(ress)))

0.0    306
1.0    204
Name: Gender_bins, dtype: int64
204
510
0.40000


## Save this as a reranking and plot the entropy and utility

In [75]:
re_ranked_WO = pd.DataFrame(ress, columns = ['Idx','Gender_bins','Preds'])
re_ranked_WO.to_csv("re_ranked_WO.csv")

# re_ranked_PLAIN = pd.DataFrame(ress, columns = ['Idx','Gender_bins','Preds'])
# re_ranked_PLAIN.to_csv("re_ranked_PLAIN.csv")